## Imports and settings

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import json
import pprint

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import ipywidgets as widgets
from matplotlib import gridspec
import pprint

detector_path = '..'
results_path = os.path.join(detector_path, 'results')
sys.path.append(detector_path)

from utils import constants
from utils import errors
from sleep.mass import MASS
from sleep.inta import INTA
from sleep import postprocessing
from sleep import data_ops
from evaluation import metrics
from evaluation import data_manipulation

SEED = 123
DPI = 200

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Load database and predicted probabilities

In [ ]:
# TODO: 1. cargar las predicciones para todas las semillas, evaluar la curva para cada semilla en VAL,
# mostrar todas las curvas superpuestas y ademas el promedio, y mostrar el PR de cada semilla.



# Select database
dataset_name = constants.MASS_NAME
# Select predictions ckpt folder

# Best ones:
# MASS: ckpt_folder = os.path.join('20190325_v2bn_fixed_loading_train_mass', 'bsf_avg')
# INTA: ckpt_folder = os.path.join('20190328_v2bn_fixed_inta_train_inta', 'bsf_avg')
ckpt_folder = os.path.join('20190325_v2bn_fixed_loading_train_mass', 'bsf_avg')
verbose = False

# Load data

# Marks are binary sequences for each page, 200 fs resolution
errors.check_valid_value(
    dataset_name, 'dataset_name',
    [constants.MASS_NAME, constants.INTA_NAME])
if dataset_name == constants.MASS_NAME:
    dataset = MASS(load_checkpoint=True)
else:
    dataset = INTA(load_checkpoint=True)

    # Get training set ids
print('Loading train set and splitting train/val')
all_train_ids = dataset.train_ids
# Split to form validation set
train_ids, val_ids = data_manipulation.split_ids_list(
    all_train_ids, seed=SEED)
print('Training set IDs:', train_ids)
print('Validation set IDs:', val_ids)
# Get test data
print('Loading test set')
test_ids = dataset.test_ids
print('Testing set IDs:', test_ids)

# Get subjects data, with the expert used for training
print('Loading signals and marks')
set_list = ['train', 'val', 'test']
x = {}
y = {}
pages = {}
x['train'], y['train'] = dataset.get_subset_data(train_ids, which_expert=1, verbose=verbose)
x['val'], y['val'] = dataset.get_subset_data(val_ids, which_expert=1, verbose=verbose)
x['test'], y['test'] = dataset.get_subset_data(test_ids, which_expert=1, verbose=verbose)
print('Loading pages')
pages['train'] = dataset.get_subset_pages(train_ids, verbose=verbose)
pages['val'] = dataset.get_subset_pages(val_ids, verbose=verbose)
pages['test'] = dataset.get_subset_pages(test_ids, verbose=verbose)

# Load predictions (probability vectors for each page), 200/factor resolution (default factor 8)
ckpt_path = os.path.abspath(os.path.join(results_path, 'predictions_%s' % dataset_name, ckpt_folder))
print('Loading predictions from %s' % ckpt_path)
y_pred = {}
for set_name in set_list:
    y_pred[set_name] = np.load(os.path.join(ckpt_path, 'y_pred_%s.npy' % set_name), allow_pickle=True)
    # Keep only class 1 probability
    y_pred[set_name] = [this_y_pred[..., 1] for this_y_pred in y_pred[set_name]]
print('Done')